Thomas Conibear - Jacobian Homework Attempt

In class and on homework we used the LJ potential and used scipy to find the minimum. However, the scipy minimize function does not use the Jacobian in its calculations so it can often take some time to calculate large programs. So in order to optimize this we need to include the correct Jacobian and optimize that to more quickly find the correct minimum potential energy.

LJ Potential Equation: V = 4 * $(\frac{1}{r^12}$ - $\frac{1}{r^6})$

First we have to break up our LJ potential equation into vector magnitude form where r=$\sqrt{x^2+y^2+z^2}$ 

as r = x$\hat{x}$+y$\hat{y}$+z$\hat{z}$

Which leads to the form:

### LJ =  4 * $(\frac{1}{\sqrt{x^2+y^2+z^2}^12}$ - $\frac{1}{\sqrt{x^2+y^2+z^2}^6})$

So now that we have it in this format with respect to three variables. We must take the Cartesian del operator of partial derivatives with respect to this new LJ equation. 

Since each variable is identical in terms of it's powers and coefficients, the partial derivatives will be identical as well.

So we have:

### LJ =  del * [4 * $(\frac{1}{\sqrt{x^2+y^2+z^2}^12}$ - $\frac{1}{\sqrt{x^2+y^2+z^2}^6})$]

Which becomes:

### $LJ_x$ =  4 * $(\frac{-6(2x)}{\sqrt{x^2+y^2+z^2}^12}$ - $\frac{3(2x)}{\sqrt{x^2+y^2+z^2}^6})$  =  4 * $(\frac{-12x}{\sqrt{x^2+y^2+z^2}^12}$ - $\frac{6x}{\sqrt{x^2+y^2+z^2}^6})$
with respect to x.

It is the same with respect to y and z. We have as follows:

Y: 
### $LJ_y$ =  4 * $(\frac{-12y}{\sqrt{x^2+y^2+z^2}^12}$ - $\frac{6y}{\sqrt{x^2+y^2+z^2}^6})$
Z:
### $LJ_z$ =  4 * $(\frac{-12z}{\sqrt{x^2+y^2+z^2}^12}$ - $\frac{6z}{\sqrt{x^2+y^2+z^2}^6})$

So our final Jacobian will be the summation of these as it is a vector sum now. 


$\sum$ $LJ_x$ + $LJ_y$ + $LJ_z$

However, this only deals with one point, since we are interested in potential between two points we must do a double summation of potentials. So we have:

$\sum$ $\sum$ $LJ_x$ + $LJ_y$ + $LJ_z$

Now we take the scipy minimize of this double summation. The attempt at coding this is below.

In [1]:
import numpy as np
from numba import jit
#Use jit to improve time of calculation

#From lecture

@jit
#LJ Potential Equation
def LJ(r):
    return 4*(1/r**12 - 1/r**6)

@jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)
    
    #positions = [x0, y0, z0, x1, y1, z1, .....  , x(n-1), y(n-1), z(n-1)] Array of many particles
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3] #Doing the array sequentially but in proper order of an atom's position
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += LJ(dist)
            return E

@jit
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

@jit
def pos_r_space(pos):
    pos_r = []
    for i in range(len(pos)//3):
        pos_r.append(np.linalg.norm(pos[i*3:(i+1)*3]))
    return pos_r

#Jacobian function in vector coordinates
@jit
def LJ_Jacobian(pos):
    N_atom = int(len(pos)/3) 
    force = np.zeros([N_atom, 3])
                 
    for i in range(N_atom):
        pos1 = pos[i*3:(i+1)*3]
        for j in range(N_atom): 
            pos2 = pos[j*3:(j+1)*3]
            if not any(pos1 == pos2): 
                dist_xyz = pos1 - pos2
                dist_r = np.linalg.norm(dist_xyz)
                r14 = np.power(dist_r, 14)
                r8 = np.power(dist_r, 8)
                next_r = np.dot((-48/r14 + 24/r8), dist_xyz)
                force[i] += next_r
    
    return force.flatten()

In [2]:
from scipy.optimize import minimize

#Energies from online website from lecture
#Finding force for n atoms up to 10 but you could add as many as you want as long as you add their respective energies
x_values = []
N_atoms = [4,5,6,7,8,9,10]
Energies = [-6.000000, -9.103852, -12.712062, -16.505384, -19.821489, -24.113360, -28.422532]
    
for n in N_atoms:
    
    fun_values = []
    f_values = 0
    count = 0
    while (f_values-Energies[n-4]>0.8 and count < 100):
        pos = init_pos(n)
        res = minimize(total_energy, pos, jac=LJ_Jacobian, method='CG', tol=1e-4)
        f_values = res.fun
        fun_values.append(res.fun)
        count +=1
        
    if (f_values-Energies[n-4]<= 0.8):
        print('\n Number of atoms = ', n, ' and the Minimum Force is',  f_values)
    else:
        print('\n Number of atoms = ', n, 'and the Minimum Force is', min(fun_values))


 Number of atoms =  4 and the Minimum Force is -0.9999999999999951

 Number of atoms =  5 and the Minimum Force is -0.9999999996356648

 Number of atoms =  6 and the Minimum Force is -0.9999999620116734

 Number of atoms =  7 and the Minimum Force is -0.9999999993689369

 Number of atoms =  8 and the Minimum Force is -0.9994051157083521

 Number of atoms =  9 and the Minimum Force is -0.9994664730923746

 Number of atoms =  10 and the Minimum Force is -0.9994716050974934
